In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize, Compose 
import matplotlib.pyplot as plt

%matplotlib inline

In [13]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args
        
    def forward(self, x):
        return x.reshape((x.shape[0],)+self.shape)

In [14]:
class ShowShape(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        print(x.shape)
        return x

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            Reshape(512*7*7),
            nn.Linear(512*7*7,1),
        )
        
    def forward(self, x):   
        return self.disc(x)     

In [16]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 7*7*256),
            Reshape(256, 7, 7),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        
    def forward(self, z):
        return self.gen(z)

In [17]:
z_dim = 1024
batch_size = 256
lr = 2e-4
num_epochs = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fixed_noise = torch.randn((100, z_dim)).to(device)

In [18]:
transforms = Compose(
    [Resize(112), ToTensor(), Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]
)
cifar = datasets.CIFAR10(root='dataset/cifar10/',transform=transforms, download=True)
loader = DataLoader(dataset=cifar, batch_size=batch_size, shuffle=True)

# dataset = datasets.ImageFolder(root='...', transform=transforms) 
# loader = DataLoader(dataset=cifar, batch_size=batch_size, shuffle=True)

In [19]:
disc = Discriminator().to(device)
gen = Generator(z_dim).to(device)

In [20]:
criterion = nn.MSELoss()
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))

In [21]:
import os
from IPython.display import Image
from torchvision.utils import save_image

sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    
    
def save_fake_img(index):
    fake_images = gen(fixed_noise)
    fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fname)
    save_image(fake_images, os.path.join(sample_dir, fname), nrow=10)

In [ ]:
import time

begin = time.time()
d_losses, g_losses = [], []
for epoch in range(0, num_epochs):
    since = time.time()
    for idx, (real, _) in  enumerate(loader):
        torch.cuda.empty_cache()
        real = real.to(device)
        train_batch_size  = real.shape[0]
        
        noise = torch.randn((train_batch_size, z_dim)).to(device)
        fake = gen(noise)
        disc_real = disc(real).reshape(-1)
        lossD_real = criterion(disc_real,torch.ones_like(disc_real))
        disc_fake= disc(fake).reshape(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        opt_disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        d_losses.append(lossD.item())

        output = disc(fake).reshape(-1)
        lossG = criterion(output, torch.ones_like(output))
        opt_gen.zero_grad()
        lossG.backward()
        opt_gen.step() 
        g_losses.append(lossG.item())
        
        if (idx+1) % 98 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}], Step [{idx+1}/{len(loader)}], Loss D: {d_losses[-1]:.4f}, loss G: {g_losses[-1]:.4f}, Time: {time.time() - since:.4f}s"
            )

    print(
            f"Total Time: {time.time() - begin:.4f}s"
        )

    save_fake_img(epoch+1)
    if (epoch)+1 % 50 == 0:
        fname = 'fake_images-{0:0=4d}.png'.format(epoch+1)
        Image(os.path.join(sample_dir, fname))

Epoch [1/100], Step [98/1563], Loss D: 0.0415, loss G: 1.5968, Time: 33.5944s


In [ ]:
torch.save(disc.state_dict(), 'D_LS.ckpt')
torch.save(gen.state_dict(), 'G_LS.ckpt')

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses')

In [ ]:
randn_noise = torch.randn((100, z_dim)).to(device)
fake = gen(randn_noise)
save_image(fake,os.path.join(sample_dir, 'test_result.png'), nrow = 10)

Image(os.path.join(sample_dir, 'test_result.png'))